In [53]:
import csv 
import gzip 
from sklearn.model_selection import cross_val_score
from sklearn import linear_model


# reading in the data to get Field Names
data = []
with open('Mortality_05_UT.csv', 'rb') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
    fieldNames = []
    for row in spamreader:
        fieldNames.append(row)
        break

In [54]:
fieldNames = fieldNames[0]

In [55]:
# create List of dictionaries for each record in the dataset
with open('Mortality_05_UT.csv') as csvfile:
    data = []
    reader = csv.DictReader(csvfile,fieldnames=fieldNames)
    count = 0 
    for row in reader:
        if count != 0:
            data.append(row)
        count +=1
        

In [56]:
# remove records that have an empty age field 
cleanData = []
for datum in data:
    toAppend = True
    if datum['age'] is '':
        toAppend = False
    for k in datum.keys():
        if datum[k] is '':
            datum[k] = -100
    if toAppend:
        cleanData.append(datum)

In [58]:
def feature(datum):
    feat = []
    for k in datum.keys():
        feat.append(datum[k])
    return feat

In [61]:
useless = ['m_id','client_m_id','hl_id','house_no','PSU_ID','m_serial_no','date_of_death','month_of_death','year_of_death','is_death_reg','place_of_death','is_death_certificate_received','serial_num_of_infant_mother','death_period','months_of_pregnancy']

In [7]:
useful_feat = ['id','state','house_hold_no','district','rural','stratum_code',\
               'deceased_sex','treatment_source','order_of_birth','sex','age',\
               'religion','marital_status','currently_attending_school', \
               'reason_for_not_attending_school','highest_qualification','occupation_status'\
              'disability_status','regular_treatment','regular_treatment_source','chew','smoke'\
              'injury_treatment_type','alcohol','house_status','owner_status','drinking_water_source'\
              'is_water_filter','water_filtration','toilet_used','is_toilet_shared','household_have_electricity'\
              'lighting_source','cooking_fuel','no_of_dwelling_rooms','kitchen_availability','is_radio',\
              'is_televison','is_computer','is_telephone', 'is_washing_machine', 'is_refrigerator', \
              'is_sewing_machine', 'is_bicyle', 'is_scooter', 'is_car', 'is_tractor', 'is_water_pump', \
              'cart', 'land_possessed', 'residancial_status', 'iscoveredbyhealthscheme', 'injury_treatment_type'\
              'house_structure']

In [8]:
useful_label =[]

In [ ]:
cleanData[10]

In [5]:
if True:
   print [1 + \
    1]

[2]
